# playground for dataprep

In [5]:
import mozilla2csv
import os
import pandas as pd

In [5]:
tsv_path = "/home/peter/Desktop/Studies/Thesis/data/cv-corpus-20.0-2024-12-06-ga-IE/cv-corpus-20.0-2024-12-06/ga-IE"
output_path = "/home/peter/Desktop/Studies/Thesis/data/cv-corpus-20.0-2024-12-06-ga-IE"

In [4]:
os.listdir(tsv_path)

['validated_sentences.tsv',
 'invalidated.tsv',
 '.~lock.train.tsv#',
 'validated.tsv',
 'clips',
 'unvalidated_sentences.tsv',
 'other.tsv',
 'clip_durations.tsv',
 'reported.tsv',
 'dev.tsv',
 'test.tsv',
 'train.tsv']

In [13]:
train = pd.read_csv(tsv_path+"/"+"train.tsv",sep="\t")
train.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,c1d05e6aa30ec2982073814640b1a5365c09e53fc52dd3...,common_voice_ga-IE_17571418.mp3,12e384c909a7793008d93b71d5b5b808372007d9463df5...,Tá a mála ar an urlár,NaN,2,1,thirties,male_masculine,Gaeilge Chonnacht,NaN,ga-IE,NaN
1,c1d05e6aa30ec2982073814640b1a5365c09e53fc52dd3...,common_voice_ga-IE_17571419.mp3,4d5c6e7d628d5a00550e4022fb1797dd41dffbc3172e42...,An Phoblacht Doiminiceach,NaN,2,0,thirties,male_masculine,Gaeilge Chonnacht,NaN,ga-IE,NaN
2,c1d05e6aa30ec2982073814640b1a5365c09e53fc52dd3...,common_voice_ga-IE_17571420.mp3,9f95ffcd6f2428b00c7ed675f307ae20788a6a618d9a08...,Do ghnó féin déan a dhuine,NaN,2,0,thirties,male_masculine,Gaeilge Chonnacht,NaN,ga-IE,NaN
3,c1d05e6aa30ec2982073814640b1a5365c09e53fc52dd3...,common_voice_ga-IE_17571424.mp3,56da7e8f95ca495e1d91db46cb4a07b21e09394090553f...,Ní raibh mé,NaN,2,1,thirties,male_masculine,Gaeilge Chonnacht,NaN,ga-IE,NaN
4,c1d05e6aa30ec2982073814640b1a5365c09e53fc52dd3...,common_voice_ga-IE_17571428.mp3,da021995eee94400c8eea98ad5db21d2eddaf2060ae9e1...,Dún do bhéal.,NaN,2,0,thirties,male_masculine,Gaeilge Chonnacht,NaN,ga-IE,NaN


In [14]:
train = train[["path","sentence"]].copy()

In [15]:
train["path"] = "audio/001/" + train["path"]

In [16]:
train.head()

,path,sentence
0,audio/001/common_voice_ga-IE_17571418.mp3,Tá a mála ar an urlár
1,audio/001/common_voice_ga-IE_17571419.mp3,An Phoblacht Doiminiceach
2,audio/001/common_voice_ga-IE_17571420.mp3,Do ghnó féin déan a dhuine
3,audio/001/common_voice_ga-IE_17571424.mp3,Ní raibh mé
4,audio/001/common_voice_ga-IE_17571428.mp3,Dún do bhéal.


In [17]:
train.to_csv(output_path+"/transcription/owsm_v3.1/001.csv")

# Finetune_owsm

In [6]:
import os
from glob import glob

import numpy as np
import librosa

import torch
from espnet2.bin.s2t_inference import Speech2Text
from espnet2.layers.create_adapter_fn import create_lora_adapter
import espnetez as ez

# Define hyper parameters
DUMP_DIR = output_path+f"/dump"
CSV_DIR = output_path+f"/transcription"
EXP_DIR = output_path+f"/exp/finetune"
STATS_DIR = output_path+f"/exp/stats_finetune"

FINETUNE_MODEL = "espnet/owsm_v3.1_ebf"
LORA_TARGET = [
    "w_1", "w_2", "merge_proj"
]
LANGUAGE = "gae"

In [9]:
pretrained_model = Speech2Text.from_pretrained(
    FINETUNE_MODEL,
    #category_sym=f"<{LANGUAGE}>",
    beam_size=10,
) # Load model to extract configs.
pretrain_config = vars(pretrained_model.s2t_train_args)
tokenizer = pretrained_model.tokenizer
converter = pretrained_model.converter
del pretrained_model

# For the configuration, please refer to the last cell in this notebook.
finetune_config = ez.config.update_finetune_config(
	's2t',
	pretrain_config,
	#f"finetune_with_lora.yaml"
)

# When you don't use yaml file, you can load finetune_config in the following way.
task_class = ez.task.get_ez_task("s2t")
default_config = task_class.get_default_config()
training_config = default_config.update(your_config_in_dict)

# define model loading function
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def freeze_parameters(model):
    for p in model.parameters():
        if p.requires_grad:
            p.requires_grad = False

def build_model_fn(args):
    pretrained_model = Speech2Text.from_pretrained(
        FINETUNE_MODEL,
        category_sym=f"<{LANGUAGE}>",
        beam_size=10,
    )
    model = pretrained_model.s2t_model
    model.train()
    print(f'Trainable parameters: {count_parameters(model)}')
    freeze_parameters(model)

    # apply lora
    create_lora_adapter(model, target_modules=LORA_TARGET)
    print(f'Trainable parameters after LORA: {count_parameters(model)}')
    return model

Fetching 37 files: 100%|██████████| 37/37 [00:00<00:00, 416056.97it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'finetune_with_lora.yaml'